In [1]:
import os
import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from src.etl import RecordETL
from src.preprocess import MLPDataPipeline, Conv1DDataPipeline
from src.models import BaseMLP,BaseConv1D
from src.train import Experiment

In [2]:
DATA_DIR = 'data/processed_data'


In [3]:
etl =  RecordETL(DATA_DIR)
raw_dataset = etl.read_from_tf_records_from_local().take(8)
test_raw_dataset = etl.read_from_tf_records_from_local().take(10).skip(8)

In [4]:
pipeline = Conv1DDataPipeline(num_seconds=10)
model = BaseConv1D()
optimizer = tf.keras.optimizers.Adam()
#need to validate on different subjects
experiment_one = Experiment(data=raw_dataset,
                            pipeline=pipeline,
                            model=model,
                            optimizer=optimizer,
                            epochs=30)




In [5]:
experiment_one.run()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/30
359/359 [==============================] - 7s 19ms/step - loss: 0.7004 - Accuracy: 0.5314 - Precision: 0.5255 - Recall: 0.5444 - val_loss: 0.6848 - val_Accuracy: 0.5432 - val_Precision: 0.5902 - val_Recall: 0.4987
Epoch 2/30
359/359 [==============================] - 5s 13ms/step - loss: 0.6863 - Accuracy: 0.5631 - Precision: 0.5579 - Recall: 0.6427 - val_loss: 0.6764 - val_Accuracy: 0.5557 - val_Precision: 0.5412 - val_Recall: 0.7645
Epoch 3/30
359/359 [==============================] - 5s 1

In [6]:
dataset = pipeline.fit_transform(raw_dataset)
test_dataset = pipeline.fit_transform(test_raw_dataset)

In [7]:
pipeline.get_dataset_length(dataset), pipeline.get_dataset_length(test_dataset)

(3588, 508)

In [8]:
X = []
y_true = []
for record in test_dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(experiment_one.model.predict(X).flatten())
(y_pred == y_true).mean()

0.6515748031496063

In [9]:
X = []
y_true = []
for record in dataset:
    X.append(record[0])
    y_true.append(record[1])
X = tf.stack(X)
y_true = tf.stack(y_true).numpy().flatten()
y_pred = np.round(experiment_one.model.predict(X).flatten())
(y_pred == y_true).mean()

0.7533444816053512